# Data cleaning Immobiliare.it

I first import the libraries that i believe they will be needed for this cleaning process.

In [187]:
import numpy as np
import pandas as pd
import re

I now need to load the ```.csv``` file into a pandas dataframe and then check what the dataframe looks like as well as its datatypes.

In [188]:
#REMEMBER TO FILL THE PROPER FILE NAME!
df_raw = pd.read_csv('house_prices_italy_2023_7.csv')
df_raw.head()

,Unnamed: 0,region,city,area,rooms,toilets,price,date
0,0,abruzzo,Pescara,295m²,5+,3+,€ 257.000,2023-07-31
1,1,abruzzo,Francavilla al Mare,88m²,3,1,€ 168.000,2023-07-31
2,2,abruzzo,Pescara,227m²,5,3+,€ 299.000,2023-07-31
3,3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000,2023-07-31
4,4,abruzzo,Francavilla al Mare,83m²,4,1,€ 139.000,2023-07-31


In [189]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  39999 non-null  int64 
 1   region      39999 non-null  object
 2   city        39999 non-null  object
 3   area        39974 non-null  object
 4   rooms       39057 non-null  object
 5   toilets     39999 non-null  object
 6   price       39999 non-null  object
 7   date        39999 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.4+ MB


As expected some columns have wrong datatypes for what I plan to do. I would like to have them numerical columns into integer format with np.nan values for null. Date column as date instead.
Let's first have a look on how many nulls we have here:

In [190]:
df_raw.isnull().sum()

Unnamed: 0      0
region          0
city            0
area           25
rooms         942
toilets         0
price           0
date            0
dtype: int64

Apart from rooms column, not so many indeed. I decided to drop the first column (*Unnamed:0*) since it does not carry any info:

In [191]:
df=df_raw.drop("Unnamed: 0", axis=1)

I change the column names to match the dimensions of the values:

In [192]:
df.rename(columns={'area':'area[m2]', 'price':'price[€]'}, inplace=True)

I want to check if the city column has been properly scaped:

In [193]:
#Remove possible white space from begin and end of the string
df['city'] = df['city'].str.strip()

#Showing the unique values
print(df['city'].value_counts().sort_values(ascending=True))
print("\nSome entries show wrongly scraped city. I need to remove them!")

#Saving them in a txt file so i can explore it better
df['city'].value_counts().sort_index(ascending=True).to_csv('city.txt', header=False)

Montagnana                                       1
Appartamenti di nuova costruzione a Genova       1
Mallare                                          1
Soveria Mannelli                                 1
Portofino                                        1
                                              ... 
Napoli                                         695
Genova                                         704
Torino                                         751
Milano                                        1121
Roma                                          1610
Name: city, Length: 3554, dtype: int64

Some entries show wrongly scraped city. I need to remove them!


In [194]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    39999 non-null  object
 1   city      39999 non-null  object
 2   area[m2]  39974 non-null  object
 3   rooms     39057 non-null  object
 4   toilets   39999 non-null  object
 5   price[€]  39999 non-null  object
 6   date      39999 non-null  object
dtypes: object(7)
memory usage: 2.1+ MB
None


In [195]:
#remove data entries from df['city] that contains the word "Appartamenti" or "Attici" or "Negozi" or "Ville"
df = df[~df['city'].str.contains("Appartamenti|Attici|Negozi|Ville")]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39490 entries, 0 to 39998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    39490 non-null  object
 1   city      39490 non-null  object
 2   area[m2]  39465 non-null  object
 3   rooms     38559 non-null  object
 4   toilets   39490 non-null  object
 5   price[€]  39490 non-null  object
 6   date      39490 non-null  object
dtypes: object(7)
memory usage: 2.4+ MB


In [196]:
#Check the result on a txt file
df['city'].value_counts().sort_index(ascending=True).to_csv('city_cleaned.txt', header=False)

Roughly 500 entries where removed because the city was not properly scraped. This is because the title did not contain the city name in that case but it was formatted differently.

Now I can start the cleaning the rest of the columns.
I force the "area" column as 'string' datatype, if not, later I noticed it will raise errors.

In [197]:
df['area[m2]'] = df['area[m2]'].astype(str)

In [198]:
p = re.compile('[0-9]+')
df['area[m2]'] = df['area[m2]'].apply(lambda x: 0 if p.search(x) is None else p.search(x).group())
df.head()

,region,city,area[m2],rooms,toilets,price[€],date
0,abruzzo,Pescara,295,5+,3+,€ 257.000,2023-07-31
1,abruzzo,Francavilla al Mare,88,3,1,€ 168.000,2023-07-31
2,abruzzo,Pescara,227,5,3+,€ 299.000,2023-07-31
4,abruzzo,Francavilla al Mare,83,4,1,€ 139.000,2023-07-31
5,abruzzo,Roseto degli Abruzzi,978,5+,3+,€ 1.500.000,2023-07-31


In [199]:
df['area[m2]'] = df['area[m2]'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39490 entries, 0 to 39998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    39490 non-null  object
 1   city      39490 non-null  object
 2   area[m2]  39490 non-null  int64 
 3   rooms     38559 non-null  object
 4   toilets   39490 non-null  object
 5   price[€]  39490 non-null  object
 6   date      39490 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.4+ MB


Since I am not particularly interested in getting to know statistics about rooms and toilets but more on prices, area and their distribution over the italian territory, I will keep **rooms** and **toilets** columns as they are, namely categorical variables of type strings.
Now for for the **price** column: 


In [200]:
df['price[€]'] = df['price[€]'].str.replace('.','', regex=False)
df['price[€]'] = df['price[€]'].str.replace('€','', regex= False)

p_price = re.compile('\d+')
df['price[€]'] = df['price[€]'].apply(lambda x: 0 if x.strip().isnumeric() == False else p.search(x).group())
df['price[€]'] = df['price[€]'].astype('int')
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 39490 entries, 0 to 39998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    39490 non-null  object
 1   city      39490 non-null  object
 2   area[m2]  39490 non-null  int64 
 3   rooms     38559 non-null  object
 4   toilets   39490 non-null  object
 5   price[€]  39490 non-null  int64 
 6   date      39490 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.4+ MB


In order to convert to integer I was forced to put *0* instead of ```np.nan```. That zero value will interfere with the statistic computation. Hence i convert it back to NaN.

In [201]:
#CONVERT 0 VALUES TO NAN
df['area[m2]'] = df['area[m2]'].apply(lambda x: np.nan if x==0 else x)
df['price[€]'] = df['price[€]'].apply(lambda x: np.nan if x==0 else x)

#CONVERT TO INT TYPE
df['area[m2]'] = df['area[m2]'].astype('Int64', errors='ignore')
df['price[€]'] = df['price[€]'].astype('Int64', errors='ignore')

df

,region,city,area[m2],rooms,toilets,price[€],date
0,abruzzo,Pescara,295,5+,3+,257000,2023-07-31
1,abruzzo,Francavilla al Mare,88,3,1,168000,2023-07-31
2,abruzzo,Pescara,227,5,3+,299000,2023-07-31
4,abruzzo,Francavilla al Mare,83,4,1,139000,2023-07-31
5,abruzzo,Roseto degli Abruzzi,978,5+,3+,1500000,2023-07-31
...,...,...,...,...,...,...,...
39994,veneto,San Zeno di Montagna,400,5+,3+,598000,2023-07-31
39995,veneto,Bassano del Grappa,260,5+,3+,440000,2023-07-31
39996,veneto,Garda,219,5+,2,1200000,2023-07-31
39997,veneto,Mirano,185,5+,2,450000,2023-07-31


In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39490 entries, 0 to 39998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    39490 non-null  object
 1   city      39490 non-null  object
 2   area[m2]  39465 non-null  Int64 
 3   rooms     38559 non-null  object
 4   toilets   39490 non-null  object
 5   price[€]  35321 non-null  Int64 
 6   date      39490 non-null  object
dtypes: Int64(2), object(5)
memory usage: 2.5+ MB


I now rename the regions into a more appropriate manner:

In [203]:
regions_dict = {'abruzzo':'Abruzzo', 'basilicata':'Basilicata', 'campania':'Campania', 'calabria':'Calabria', 'emilia-romagna':'Emilia Romagna',
       'friuli-venezia-giulia':'Friuli-Venezia Giulia', 'lazio': 'Lazio', 'liguria':'Liguria', 'lombardia':'Lombardia', 'marche':'Marche',
       'molise':'Molise', 'piemonte':'Piemonte', 'puglia':'Puglia', 'sardegna':'Sardegna', 'sicilia':'Sicilia', 'toscana':'Toscana',
       'trentino-alto-adige': 'Trentino-Alto Adige', 'umbria':'Umbria', 'valle-d-aosta':'Valle d\'Aosta', 'veneto':'Veneto'}

df.replace({'region':regions_dict}, inplace=True)
df.isna().sum()


region         0
city           0
area[m2]      25
rooms        931
toilets        0
price[€]    4169
date           0
dtype: int64

In [204]:
df.sample(20)

,region,city,area[m2],rooms,toilets,price[€],date
8337,Emilia Romagna,Bologna,105,4,2,429000,2023-07-31
32063,Trentino-Alto Adige,Bolzano,190,4,1,995000,2023-07-31
19186,Marche,Montecosaro,79,3,1,130000,2023-07-31
26930,Sardegna,Alghero,846,5+,1,5650000,2023-07-31
35111,Umbria,Gubbio,147,5+,1,<NA>,2023-07-31
10668,Friuli-Venezia Giulia,Lignano Sabbiadoro,270,4,2,430000,2023-07-31
38443,Veneto,Vigonza,190,5+,3,465000,2023-07-31
2270,Basilicata,Maratea,78,4,3,185000,2023-07-31
31552,Toscana,Pontassieve,850,5+,3+,1500000,2023-07-31
32808,Trentino-Alto Adige,Trento,61,3,1,280000,2023-07-31


I believe this is enough. I do not convert the **date** column into DateTime object since for Tableau it is perfectly capable of treating as such as it is. It will now be possible for me to work on a Dashboard.

I save the csv file:

In [205]:
year = 2023
month = 7

In [206]:
df.to_csv(f'house_prices_italy_{year}_{month}_cleaned.csv')